## 0. Download dataset

In [1]:
!wget https://motchallenge.net/data/MOT17.zip

--2024-03-21 07:45:18--  https://motchallenge.net/data/MOT17.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5860214001 (5.5G) [application/zip]
Saving to: ‘MOT17.zip’

MOT17.zip           100%[===================>]   5.46G  17.2MB/s    in 5m 4s   

2024-03-21 07:50:22 (18.4 MB/s) - ‘MOT17.zip’ saved [5860214001/5860214001]



In [2]:
!unzip -qq MOT17.zip

## 1. Import libraries

In [3]:
!pip install -q ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 722.0/722.0 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import os
import yaml
import shutil
import configparser
import ultralytics
ultralytics.checks()

from tqdm import tqdm
from ultralytics import YOLO

Ultralytics YOLOv8.1.30 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 40.0/78.2 GB disk)


## 2. Convert to YOLO format

In [5]:
def convert_to_yolo_format(bb, img_width, img_height):
    x_center = bb['bb_left'] + (bb['bb_width'] / 2)
    y_center = bb['bb_top'] + (bb['bb_height'] / 2)

    # Normalize the coordinates by the dimensions of the image
    x_center /= img_width
    y_center /= img_height
    bb_width_normalized = bb['bb_width'] / img_width
    bb_height_normalized = bb['bb_height'] / img_height

    # Clip the values to make sure they are between 0 and 1
    x_center = max(min(x_center, 1), 0)
    y_center = max(min(y_center, 1), 0)
    bb_width_normalized = max(min(bb_width_normalized, 1), 0)
    bb_height_normalized = max(min(bb_height_normalized, 1), 0)

    return (x_center, y_center, bb_width_normalized, bb_height_normalized)

In [6]:
def process_folder(folder_path):
    # Read image dimensions from seqinfo.ini
    config = configparser.ConfigParser()
    config.read(os.path.join(folder_path, 'seqinfo.ini'))
    img_width = int(config['Sequence']['imWidth'])
    img_height = int(config['Sequence']['imHeight'])

    # Load ground truth data
    gt_path = os.path.join(folder_path, 'det/det.txt')
    gt_data = pd.read_csv(
        gt_path,
        header=None,
        names=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'class', 'visibility']
    )

    labels_folder = os.path.join(folder_path, 'labels')
    os.makedirs(labels_folder, exist_ok=True)

    for frame_number in gt_data['frame'].unique():
        frame_data = gt_data[gt_data['frame'] == frame_number]
        label_file = os.path.join(labels_folder, f'{frame_number:06d}.txt')

        with open(label_file, 'w') as file:
            for _, row in frame_data.iterrows():
                yolo_bb = convert_to_yolo_format(row, img_width, img_height)
                file.write(f'0 {yolo_bb[0]} {yolo_bb[1]} {yolo_bb[2]} {yolo_bb[3]}\n')

In [7]:
def process_all_folders(base_directory):
    # List all subdirectories in the base directory
    for folder_name in tqdm(os.listdir(base_directory)):
        folder_path = os.path.join(base_directory, folder_name)

        # Delete folder not contain 'FRCNN' in name
        if 'FRCNN' not in folder_name:
            os.system(f'rm -rf {folder_path}')
            continue

        if os.path.isdir(folder_path):
            process_folder(folder_path)

In [8]:
process_all_folders('MOT17/train')
process_all_folders('MOT17/test')

100%|██████████| 21/21 [00:10<00:00,  1.97it/s]


## 3. Move file

In [9]:
def rename_and_move_files(src_folder, dst_folder, folder_name, file_extension):
    for filename in os.listdir(src_folder):
        if filename.endswith(file_extension):
            # Include folder name in the new filename
            new_filename = f'{folder_name}_{filename}'
            shutil.move(os.path.join(src_folder, filename), os.path.join(dst_folder, new_filename))

In [10]:
def move_files_all_folders(base_directory):
    images_dir = os.path.join(base_directory, 'images')
    labels_dir = os.path.join(base_directory, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for folder_name in tqdm(os.listdir(base_directory)):
        if folder_name in ['images', 'labels']:  # Skip these folders
            continue

        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path):
            rename_and_move_files(os.path.join(folder_path, 'img1'), images_dir, folder_name, '.jpg')
            rename_and_move_files(os.path.join(folder_path, 'labels'), labels_dir, folder_name, '.txt')

In [11]:
move_files_all_folders('MOT17/train')
move_files_all_folders('MOT17/test')

100%|██████████| 9/9 [00:00<00:00, 13.88it/s]


In [12]:
def delete_subfolders(base_directory):
    for folder_name in os.listdir(base_directory):
        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path) and folder_name not in ['images', 'labels']:
            shutil.rmtree(folder_path)
            print(f"Deleted folder: {folder_name}")

In [13]:
# Delete all subfolders except 'images' and 'labels'
delete_subfolders('MOT17/train')
delete_subfolders('MOT17/test')

Deleted folder: MOT17-09-FRCNN
Deleted folder: MOT17-13-FRCNN
Deleted folder: MOT17-10-FRCNN
Deleted folder: MOT17-04-FRCNN
Deleted folder: MOT17-11-FRCNN
Deleted folder: MOT17-02-FRCNN
Deleted folder: MOT17-05-FRCNN
Deleted folder: MOT17-01-FRCNN
Deleted folder: MOT17-08-FRCNN
Deleted folder: MOT17-03-FRCNN
Deleted folder: MOT17-06-FRCNN
Deleted folder: MOT17-07-FRCNN
Deleted folder: MOT17-14-FRCNN
Deleted folder: MOT17-12-FRCNN


## 4. Create yaml file

In [14]:
class_labels = [
    'objects'
]
dataset_root_dir = os.path.join(
    os.getcwd(),
    'MOT17'
)
yolo_yaml_path = os.path.join(
    dataset_root_dir,
    'mot17_data.yml'
)

data_yaml = {
    'path': dataset_root_dir,
    'train': 'train/images',
    'val': 'test/images',
    'nc': len(class_labels),
    'names': class_labels
}

with open(yolo_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

## 5. Training

In [16]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')

# Config
epochs = 10
batch_size = -1 # Auto scale based on GPU memory
img_size = 640
project_name = 'models/yolo'
name = 'yolov8s_mot17_det'

# Train the model
results = model.train(
    data=yolo_yaml_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=project_name,
    name=name
)

Ultralytics YOLOv8.1.30 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/MOT17/mot17_data.yml, epochs=10, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=models/yolo, name=yolov8s_mot17_det2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True

train: Scanning /content/MOT17/train/labels.cache... 5316 images, 0 backgrounds, 0 corrupt: 100%|██████████| 5316/5316 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/MOT17/test/labels.cache... 5908 images, 11 backgrounds, 0 corrupt: 100%|██████████| 5919/5919 [00:00<?, ?it/s]


Plotting labels to models/yolo/yolov8s_mot17_det2/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00044531249999999996), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to models/yolo/yolov8s_mot17_det2
Starting training for 10 epochs...
Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       5919     110141      0.905      0.813      0.906      0.635

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      10/10       5.7G     0.6777     0.3834     0.8484        223        640: 100%|██████████| 280/280 [03:17<00:00,  1.42it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 156/156 [02:19<00:00,  1.12it/s]


                   all       5919     110141      0.903      0.812      0.914      0.642

10 epochs completed in 0.950 hours.
Optimizer stripped from models/yolo/yolov8s_mot17_det2/weights/last.pt, 22.5MB
Optimizer stripped from models/yolo/yolov8s_mot17_det2/weights/best.pt, 22.5MB

Validating models/yolo/yolov8s_mot17_det2/weights/best.pt...
Ultralytics YOLOv8.1.30 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 156/156 [02:34<00:00,  1.01it/s]


                   all       5919     110141      0.902      0.812      0.914      0.642
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to models/yolo/yolov8s_mot17_det2


## 6. Evaluation

In [17]:
from ultralytics import YOLO

# Load the trained model
model_path = os.path.join(
    project_name, name, 'weights/best.pt'
)
model = YOLO(model_path)

metrics = model.val(
    project=project_name,
    name='detect/val'
)

Ultralytics YOLOv8.1.30 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/MOT17/test/labels.cache... 5908 images, 11 backgrounds, 0 corrupt: 100%|██████████| 5919/5919 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 370/370 [02:45<00:00,  2.23it/s]


                   all       5919     110141      0.889      0.811      0.905      0.605
Speed: 0.2ms preprocess, 5.9ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to models/yolo/detect/val


## 7. Inference

In [18]:
sample_path = 'MOT17/test/images/MOT17-01-FRCNN_000001.jpg'
results = model.predict(
    sample_path,
    project=project_name,
    name='detect/predict',
    save=True
)


image 1/1 /content/MOT17/test/images/MOT17-01-FRCNN_000001.jpg: 384x640 10 objectss, 157.1ms
Speed: 3.5ms preprocess, 157.1ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)
Results saved to models/yolo/detect/predict
